In [1]:
import torch

from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import ConcatDataset, DataLoader, random_split

In [2]:
from models_by_hand import GCNConvByHand
from helpers import CVFConfigForGCNWSuccWEIDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
EPS = 1e-10

In [5]:
dataset_s_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_n7_config_rank_dataset.csv",
    "star_graph_n7_edge_index.json",
)

dataset_rr_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_random_regular_graph_n7_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n7_d4_edge_index.json",
)

dataset_plc_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n7_edge_index.json",
)

dataset_s_2_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_2_n7_config_rank_dataset.csv",
    "star_graph_2_n7_edge_index.json",
)

dataset_s_3_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_2_n7_config_rank_dataset.csv",
    "star_graph_2_n7_edge_index.json",
)


batch_size = 64

dataset_coll = [
    dataset_s_n7,
    # dataset_s_2_n7,
    # dataset_s_3_n7,
    dataset_rr_n7,
    dataset_plc_n7,
]

In [6]:
train_sizes = [int(0.95 * len(ds)) for ds in dataset_coll]
test_sizes = [len(ds) - trs for ds, trs in zip(dataset_coll, train_sizes)]

train_test_datasets = [
    random_split(ds, [tr_s, ts])
    for ds, tr_s, ts in zip(dataset_coll, train_sizes, test_sizes)
]

train_datasets = [ds[0] for ds in train_test_datasets]
test_datasets = [ds[1] for ds in train_test_datasets]

In [7]:
datasets = ConcatDataset(train_datasets)
print(f"Train Dataset size: {len(datasets):,}")

Train Dataset size: 100,293


In [8]:
dataloader = DataLoader(datasets, batch_size=batch_size, shuffle=True)

In [9]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConvByHand(dim_in, dim_h, bias=False, device=device)
        self.gcn2 = GCNConvByHand(dim_h, dim_h, bias=True, device=device)
        # self.linear1 = torch.nn.Linear(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    @property
    def edge_index(self):
        return self._edge_index

    @edge_index.setter
    def edge_index(self, val):
        self._edge_index = val

    def forward(self, x, A):
        h = self.gcn1(x, A)
        h = torch.relu(h)
        h = self.gcn2(h, A)
        h = torch.relu(h)
        # h = self.linear1(h)
        # h = torch.relu(h)
        h = self.out(h)
        h = torch.relu(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.0001)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            relative_loss = 0
            for batch in dataloader:
                x = batch[0]
                y = batch[1]
                y = y.unsqueeze(-1)
                optimizer.zero_grad()
                out = self(x[0], x[1])
                loss = criterion(out, y)
                relative_loss += torch.sum(abs(y - out) / (y + EPS))
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print(
                "Training set | Epoch",
                epoch,
                "| MSE Loss:",
                round((total_loss / count).item(), 4),
                "| Relative Loss:",
                round((relative_loss / count).item(), 4),
            )

In [10]:
gnn = VanillaGNN(3, 64, 1).to(device)
print(gnn)
print()
print("Total parameters:", f"{sum(p.numel() for p in gnn.parameters()):,}")
print()
gnn.fit(epochs=20)

VanillaGNN(
  (gcn1): GCNConvByHand(
    (linear): Linear(in_features=3, out_features=64, bias=False)
  )
  (gcn2): GCNConvByHand(
    (linear): Linear(in_features=64, out_features=64, bias=True)
  )
  (out): Linear(in_features=64, out_features=1, bias=True)
)

Total parameters: 4,417

Training set | Epoch 1 | MSE Loss: 0.2067 | Relative Loss: 514716320.0
Training set | Epoch 2 | MSE Loss: 0.0939 | Relative Loss: 132556696.0
Training set | Epoch 3 | MSE Loss: 0.0917 | Relative Loss: 134557360.0
Training set | Epoch 4 | MSE Loss: 0.0916 | Relative Loss: 148653792.0
Training set | Epoch 5 | MSE Loss: 0.0881 | Relative Loss: 127980336.0
Training set | Epoch 6 | MSE Loss: 0.087 | Relative Loss: 124421464.0
Training set | Epoch 7 | MSE Loss: 0.0857 | Relative Loss: 91598488.0
Training set | Epoch 8 | MSE Loss: 0.0847 | Relative Loss: 80778848.0
Training set | Epoch 9 | MSE Loss: 0.0841 | Relative Loss: 80908560.0
Training set | Epoch 10 | MSE Loss: 0.0824 | Relative Loss: 69113560.0
Trainin

# Testing

In [11]:
import csv

f = open("test_result_w_succ_same_nodes.csv", "w", newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["Dataset", "Actual", "Predicted"])

criterion = torch.nn.MSELoss()

gnn.eval()

with torch.no_grad():
    test_concat_datasets = ConcatDataset(test_datasets)
    test_dataloader = DataLoader(
        test_concat_datasets, batch_size=len(test_concat_datasets)
    )
    total_loss = 0
    relative_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out = gnn(x[0], x[1])
        csv_writer.writerows(
            (i, j.item(), k.item())
            for (i, j, k) in zip(
                x[2], y.detach().cpu().numpy(), out.detach().cpu().numpy()
            )
        )
        loss = criterion(out, y)
        # print("Loss: ", loss)
        total_loss += loss
        relative_loss += torch.sum(abs(y - out) / (y + EPS))
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Relative Loss:",
        round((relative_loss / count).item(), 4),
        "| Total matched",
        total_matched,
        "out of",
        len(test_concat_datasets),
        f"(Accuracy: {round(total_matched/len(test_concat_datasets) * 100, 2)}%)",
    )

f.close()

Test set | MSE loss: 0.076 | Relative Loss: 2124556416.0 | Total matched 4752 out of 5280 (Accuracy: 90.0%)


# Testing with Untrained Datasets

In [12]:
dataset_rr_n8 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_random_regular_graph_n8_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n8_d4_edge_index.json",
)

dataset_plc_n9 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n9_edge_index.json",
)

dataset_plc_n5 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n5_edge_index.json",
)

# dataset_s_3_n7 = CVFConfigForGCNWSuccWEIDataset(
#     device,
#     "star_graph_3_n7_config_rank_dataset.csv",
#     "star_graph_3_n7_edge_index.json",
# )

dataset_s_2_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_2_n7_config_rank_dataset.csv",
    "star_graph_2_n7_edge_index.json",
)


dataset = dataset_s_2_n7

with torch.no_grad():
    test_dataloader = DataLoader(dataset, batch_size=10240)
    total_loss = 0
    relative_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out = gnn(x[0], x[1])
        loss = criterion(out, y)
        total_loss += loss
        relative_loss += torch.sum(abs(y - out) / (y + EPS))
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        f"| {dataset.dataset_name}",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Relative Loss:",
        round((relative_loss / count).item(), 4),
        "| Total matched",
        f"{total_matched:,}",
        "out of",
        f"{len(dataset):,}",
        f"(Accuracy: {round(total_matched/len(dataset) * 100, 2)}%)",
    )

Test set | star_graph_2_n7 | MSE loss: 0.0654 | Relative Loss: 39041105920.0 | Total matched 406 out of 448 (Accuracy: 90.62%)
